In [ ]:
# Importing required libraries
import os #imported for file management purposes like loading the dataset etc
import random # for generating random numbers
import numpy as np # for numerical computations
import tensorflow as tf # for performing deep learning operations
from keras.preprocessing import image #  imported Keras for building and training the CNN models
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.callbacks import ModelCheckpoint
from sklearn.metrics import classification_report, confusion_matrix, precision_recall_fscore_support, precision_score, recall_score, f1_score
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Input
import matplotlib.pyplot as plt # imported for plotting the training/validation accuracy plots

from sklearn.metrics import roc_curve, roc_auc_score

In [ ]:
# Setting the random seeds for the reproducibility of the results
# The segments of the code were taken from
# https://keras.io/api/utils/python_utils/#set_random_seed-function
# https://github.com/keras-team/keras/blob/f6c4ac55692c132cd16211f4877fac6dbeead749/keras/src/applications/vgg16.py#L20-L226

tf.random.set_seed(5)
np.random.seed(5)
random.seed(5)

In [ ]:
# Defining IVGG13 Model Architecture
# The main code was taken from https://github.com/keras-team/keras/blob/f6c4ac55692c132cd16211f4877fac6dbeead749/keras/src/applications/vgg16.py#L20-L226
# and inspiration of making modifications in the code were taken from https://pysource.com/2022/10/04/vgg16-from-scratch-computer-vision-with-keras-p-7/

def IVGG13(input_shape, num_classes):

     # Input layer
    inputs = Input(shape=input_shape)
    
    # Block 1
    x = Conv2D(32, (3, 3), activation='relu', padding='same', name='block1_conv1')(inputs)
    x = Conv2D(32, (3, 3), activation='relu', padding='same', name='block1_conv2')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block1_pool')(x)

    # Block 2
    x = Conv2D(32, (3, 3), activation='relu', padding='same', name='block2_conv1')(x)
    x = Conv2D(32, (3, 3), activation='relu', padding='same', name='block2_conv2')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block2_pool')(x)

    # Block 3
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block3_conv1')(x)
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block3_conv2')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block3_pool')(x)

    # Block 4
    x = Conv2D(128, (3, 3), activation='relu', padding='same', name='block4_conv1')(x)
    x = Conv2D(128, (3, 3), activation='relu', padding='same', name='block4_conv2')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block4_pool')(x)

    # Block 5
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block5_conv1')(x)
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block5_conv2')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block5_pool')(x)

    # Classification block
    x = Flatten(name='flatten')(x)
    x = Dense(1024, activation='relu', name='fc1')(x)
    x = Dense(1024, activation='relu', name='fc2')(x)
    outputs = Dense(num_classes, activation='sigmoid', name='predictions')(x)

    # Creating model
    model = Model(inputs, outputs)
    return model

In [ ]:
# Defining input image shape
input_shape = (128, 128, 3)

# Number of classes in our dataset i.e. Normal or Pneumonia
num_classes = 1 # This line indicates that the output layer should have 1 neuron (since it is a binary classification).

model = IVGG13(input_shape=input_shape, num_classes=num_classes)
# building the neural network using the specified design provided by the function.
# Once created, this model is further used in the training and evaluation process.

In [ ]:
# Compiling the model and setting up the hyperparameters (same as mentioned in the research paper)
# https://keras.io/api/optimizers/

learning_rate = 0.001
epochs = 60
batch_size = 64
optimizer = Adam(learning_rate=learning_rate)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

# (loss='binary_crossentropy')Specifies the loss function to be minimized during training for binary classification.
# (metrics=['accuracy']) Tracks the accuracy metric during training to evaluate model performance.
# Adam optimizer is an optimization algorithm used to minimize the loss function during the training of the model

In [ ]:
# Printing the model summary
model.summary()

In [ ]:
# Setting up the directories for training, validation, and testing sets

train_dir = 'dataset/aug2/train'
validation_dir = 'dataset/aug2/val'
test_dir = 'dataset/aug2/test'

In [ ]:
# Loads the images from the specified directories into TensorFlow datasets.
# Images are resized to 128x128 pixels, and the labels are inferred from the directory structure.
# This function from TensorFlow's Keras API is used to load images from a directory and create an object,
# which is an efficient and scalable way to handle data pipelines in TensorFlow.
# for loading images is highly efficient because TensorFlow manages the loading and preprocessing in parallel, which speeds up the training process


# https://keras.io/api/data_loading/
# https://keras.io/examples/vision/image_classification_from_scratch/
# https://keras.io/examples/vision/super_resolution_sub_pixel/



train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    train_dir, # Reads images from the specified directories
    labels='inferred', # Automatically labels the images based on the directory structure (e.g., "Normal" as 0 and "Pneumonia" as 1).
    label_mode='binary', # Specifies the type of label for the output dataset means each label will be either 0 or 1 in our case.
    batch_size=batch_size, # The number of images to be loaded and processed in each batch.
    image_size=(128, 128) # Resizes all images to 128x128 pixels, which is required for consistent input to the neural network
)

validation_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    validation_dir,
    labels='inferred',
    label_mode='binary',
    batch_size=batch_size,
    image_size=(128, 128)
)

test_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    test_dir,
    labels='inferred',
    label_mode='binary',
    batch_size=batch_size,
    image_size=(128, 128),
    shuffle=False
)

In [ ]:
# Defining a function to normalize the image pixel values to the range [0, 1] by dividing by 255.
# It is a preprocessing step which scales the pixel values of images to a specific range, which improves the efficiency and performance of the model.

# tf.cast(image, tf.float32) converts the image data type to tf.float32 (32-bit floating-point).
# This is necessary because the pixel values are often loaded as integers (e.g., 0 to 255), 
# and floating-point precision is required for mathematical operations and model training.
# Divides each pixel value by 255. Since pixel values range from 0 to 255, dividing by 255 scales them to the range 0,1.
# For example, if a pixel has a value of 128, the normalization would convert it to 128/255 = 0.50196


def normalize(image, label):
    image = tf.cast(image, tf.float32) / 255.0
    return image, label

# Applying normalization to datasets
# https://keras.io/keras_cv/
# https://keras.io/examples/vision/image_classification_from_scratch

# The .map() method applies the normalize function to all images in the dataset, ensuring that all input data is normalized before training the model.
# (num_parallel_calls=tf.data.AUTOTUNE) tells TensorFlow to dynamically decide the number of parallel calls based on available resources.
# This improves performance by parallelizing the data preprocessing step.
train_dataset = train_dataset.map(normalize, num_parallel_calls=tf.data.AUTOTUNE) 
validation_dataset = validation_dataset.map(normalize, num_parallel_calls=tf.data.AUTOTUNE)
test_dataset = test_dataset.map(normalize, num_parallel_calls=tf.data.AUTOTUNE)


# repeat() Repeating the datasets to ensure that the dataset is available for multiple epochs without interruption.
# shuffle(): Randomizing the order of the data to prevent learning from the sequence of the data.
# shuffle(1000): Randomly shuffles the data with a buffer size of 1000 which prevents the model from learning any patterns related to the order.
# prefetch(): Optimizes the data pipeline by preparing the next batch of data in the background, reducing latency and improving training speed.

# prefetch(buffer_size=tf.data.AUTOTUNE): Prefetches the next batch of data while the model is training on the current batch,
# thus optimizing the input pipeline means repeat, shuffle, and prefetch operations.

# By using these techniques, we are ensuring that the model has a steady flow of data, learns effectively
# without overfitting to any order, and trains or evaluates as quickly as possible.

train_dataset = train_dataset.repeat().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
validation_dataset = validation_dataset.repeat().prefetch(buffer_size=tf.data.AUTOTUNE)
test_dataset = test_dataset.prefetch(buffer_size=tf.data.AUTOTUNE)

In [ ]:
# https://keras.io/examples/vision/image_classification_from_scratch/

# Defining the checkpoint callback to save the best model (based on validation accuracy) during training to a specified file.

# verbose = 1 controls the verbosity mode. When set to 1, it means it will print messages to the console every time the model is saved,
# providing feedback during training.

# save_best_only=True ensures that only the best model, based on the validation metric is saved.
# The model is only saved if the monitored metric (val_accuracy) improves from the previous best value.
# mode = max means the model will be saved only if the val_accuracy increases mean improves.
checkpoint_filepath = 'best_model/withROC.keras'
checkpoint = ModelCheckpoint(checkpoint_filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')

In [ ]:
# Training the model
# https://keras.io/examples/vision/image_classification_from_scratch/
# https://keras.io/examples/vision/xray_classification_with_tpus/

history = model.fit(
    train_dataset,
    steps_per_epoch=train_steps_per_epoch,
    epochs=epochs,
    validation_data=validation_dataset,
    validation_steps=validation_steps,
    callbacks=[checkpoint]
)

In [ ]:
# Evaluating the model on the test data
# https://keras.io/examples/vision/cutmix/

# The f before the quotation marks indicates that this is an f-string, a way to embed expressions inside string literals, using curly braces.
# f means floating-point number.
# .2 means two decimal places.
# :.2f is a format specifier, it means it will round the number to two decimal places, resulting in 0.96.
test_loss, test_acc = model.evaluate(test_dataset)
print(f"Test accuracy: {test_acc:.2f}") 

In [ ]:
# # Collects the true labels and model predictions from the test dataset for further evaluation.
y_true = [] # A list to store the true labels from the test dataset.
y_pred = [] # A list to store the model's predictions for the test dataset.

# iterating over the test dataset to the retrieve a specific number of batches (test_steps = 15) from the test dataset.
# It ensures that we are evaluating the model over a defined number of samples (1000 / 64 = 15)

# images: The images in the current batch.
# labels: The true labels corresponding to the images in the current batch.

# labels.numpy() means converting the labels to a NumPy array
# y_true.extend(): Appends the true labels from the current batch to the y_true list.

for images, labels in test_dataset:
    y_true.extend(labels.numpy()) 
    y_pred.extend(model.predict(images) > 0.5)  # using the trained model to generate predictions for the loaded images

y_true = np.array(y_true) # Converting the y_true list to a NumPy array for easier analysis.
y_pred = np.array(y_pred).astype(int) #Converting the y_pred list to a NumPy array and ensuring that the values are integers.



fpr, tpr, thresholds = roc_curve(y_true, y_pred)  # Calculate FPR, TPR, and thresholds
auc_score = roc_auc_score(y_true, y_pred) # Calculate AUC
print(f"AUC Score: {auc_score}")

# Plotting ROC curve


In [ ]:
# Print confusion matrix and classification report
print('Confusion Matrix')
print(confusion_matrix(y_true, y_pred))

print('Classification Report')
target_names = ['NORMAL', 'PNEUMONIA']
print(classification_report(y_true, y_pred, target_names=target_names))

# Plot the confusion matrix and capture the axis
fig, ax = plt.subplots(figsize=(6, 4))
disp.plot(cmap=plt.cm.Blues, ax=ax, values_format='d')

# Customize title
plt.title("IVGG13", fontweight='bold', fontsize=14)

# Customize axis labels
ax.set_xlabel("Predicted Label", fontsize=12)
ax.set_ylabel("True Label", fontsize=12)
ax.tick_params(axis='both', labelsize=12) 

# Customize text inside the boxes
for text in disp.text_.ravel():
    text.set_fontsize(12)   
    
# Save and show
plt.tight_layout()
plt.savefig('confusion_matrix.png', dpi=300)
plt.show()


In [ ]:
precision = precision_score(y_true, y_pred)
recall = recall_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred)

# Print results
print(f"Precision: {precision:.4f}")
print(f"Recall:    {recall:.4f}")
print(f"F1-Score:  {f1:.4f}")

In [ ]:
# Plotting training and validation accuracy
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

# Save the plot as a PNG file
plt.savefig('training_validation_accuracy.png')  
plt.show()

In [ ]:
# Plotting training and validation loss
plt.figure()
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.xlim([0, 60])  # Setting x-axis range from 0 to 60
plt.ylim([0, 3.5])   # Setting y-axis range from 0 to 5
# Save the plot as a PNG file
plt.savefig('training_validation_loss.png')  
plt.show()



In [ ]:
# Plot ROC curve

plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, label=f"AUC = {auc_score:.2f}")
plt.plot([0, 1], [0, 1], linestyle='--', color='gray')  # Diagonal line
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC Curve")
plt.legend(loc="lower right")
plt.savefig('RoC.png')
plt.show()

In [ ]:
# Saving true labels and predicted probabilities for IVGG13
np.save('y_true_model1.npy', y_true)
np.save('y_pred_proba_model1.npy', y_pred)